In [1]:
# from glob import glob
# import datetime as dt
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')

In [2]:
import xarray as xr
import json
import pandas as pd
import numpy as np
import useful_functions as uf

In [3]:
params = json.load(open(
    os.path.join('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/data',
                 'dbpm_size_params.json')))

In [4]:
dbpm_input = pd.read_parquet(
    os.path.join('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/data',
                 'dbpm_clim-fish-inputs_fao-58_1841-2010.parquet'))
dbpm_input = dbpm_input[dbpm_input.year == 1960].reset_index(drop = True)

In [5]:
test = uf.sizemodel(params, dbpm_input)

In [6]:
test['catch_pred'].sum('size_class')

<xarray.DataArray (time: 13)> Size: 104B
array([0.00000000e+00, 3.37003797e-17, 6.39878159e-16, 4.21908864e-15,
       1.69953360e-14, 5.05138377e-14, 1.24529241e-13, 2.71017931e-13,
       5.42603200e-13, 1.02434285e-12, 1.86487002e-12, 3.39292477e-12,
       6.52751571e-12])
Coordinates:
  * time     (time) datetime64[ns] 104B 1959-12-01 1960-01-01 ... 1960-12-01

In [6]:
test['catch_pred'].sum('size_class')

<xarray.DataArray (time: 13)> Size: 104B
array([0.00000000e+00, 3.37003797e-17, 6.39878159e-16, 4.21908864e-15,
       1.69953360e-14, 5.05138377e-14, 1.24529241e-13, 2.71017931e-13,
       5.42603200e-13, 1.02434285e-12, 1.86487002e-12, 3.39292477e-12,
       6.52751571e-12])
Coordinates:
  * time     (time) datetime64[ns] 104B 1959-12-01 1960-01-01 ... 1960-12-01

## Parameters from `sizemodel` function

In [12]:
ERSEM_det_input = False
temp_effect = True
use_init = False

In [13]:
[ind_min_detritivore_size] = params['ind_min_detritivore_size'] 
[numb_size_bins] = params['numb_size_bins']
[numb_time_steps] = params['numb_time_steps']
[timesteps_years] = params['timesteps_years']
idx_new = np.arange(ind_min_detritivore_size, numb_size_bins)
#Size bin index
size_bin_index = np.arange(0, numb_size_bins)
log10_size_bins = np.array(params['log10_size_bins'])
log10_size_bins_mat = xr.DataArray(data = log10_size_bins, 
                               dims = ['size_class'], 
                               coords = {'size_class': log10_size_bins})
size_bin_vals = 10**log10_size_bins_mat
[log_size_increase] = params['log_size_increase']
#Time
time = np.array(pd.date_range(dbpm_input.time.min()-pd.DateOffset(months = 1),
                                  dbpm_input.time.max(), freq = 'MS'),
                    dtype = 'datetime64[ns]')
dbpm_input_time = np.array(dbpm_input.time, dtype = 'datetime64[ns]')
effort = xr.DataArray(data = params['effort'], dims = 'time', 
                          coords = {'time': dbpm_input_time})
[log10_pred_prey_ratio] = params['log10_pred_prey_ratio']
[log_prey_pref] = params['log_prey_pref']
[metabolic_req_pred] = params['metabolic_req_pred']
[metabolic_req_detritivore] = params['metabolic_req_detritivore']
#Index for minimum detritivore size
[ind_min_detritivore_size] = params['ind_min_detritivore_size']
#Index for minimum predator size
[ind_min_pred_size] = params['ind_min_pred_size']
#Index for minimum size of detritivore fished
[ind_min_fish_det] = params['ind_min_fish_det']
#Index for minimum size of predator fished
[ind_min_fish_pred] = params['ind_min_fish_pred']

#Sinking rate
sinking_rate = xr.DataArray(data = params['sinking_rate'], dims = 'time', 
                            coords = {'time': dbpm_input_time})

In [14]:
slope_phy_zoo_mat = xr.DataArray(data = np.array((params['slope_phy_zoo']*
                                                  numb_size_bins)).\
                                 reshape(numb_size_bins, numb_time_steps),
                                 dims = ['size_class', 'time'], 
                                 coords = {'size_class': log10_size_bins,
                                           'time': dbpm_input_time})

In [15]:
reprod_det = uf.init_da(log10_size_bins, time)
reprod_pred = uf.init_da(log10_size_bins, time)
growth_det = uf.init_da(log10_size_bins, time)
growth_int_pred = uf.init_da(log10_size_bins, time)
pred_mort_det = uf.init_da(log10_size_bins, time)
pred_mort_pred = uf.init_da(log10_size_bins, time)
tot_mort_det = uf.init_da(log10_size_bins, time)
tot_mort_pred = uf.init_da(log10_size_bins, time)

In [16]:
constant_growth = uf.gphi_f(uf.pred_prey_matrix(log10_size_bins), 
                            log10_pred_prey_ratio, log_prey_pref)
constant_mortality = uf.mphi_f(-uf.pred_prey_matrix(log10_size_bins), 
                               log10_pred_prey_ratio, log_prey_pref, metabolic_req_pred)
met_req_log10_size_bins = uf.expax_f(log10_size_bins_mat, metabolic_req_pred)
ui0 = xr.DataArray(data = 10**np.array(params['int_phy_zoo']*numb_size_bins).\
                   reshape(numb_size_bins, numb_time_steps),
                   dims = ['size_class', 'time'], 
                   coords = {'size_class': log10_size_bins,
                             'time': dbpm_input_time})

In [17]:
# set up with the initial values from param
predators = ui0*(10**(slope_phy_zoo_mat*log10_size_bins_mat))
predators = xr.where(predators.size_class < log10_size_bins[ind_min_pred_size-1],
                     predators, 0)
predators.loc[{'size_class': log10_size_bins[(ind_min_pred_size-1):120],
               'time': predators.time.min()}] = params['plank_pred_sizes'][(ind_min_pred_size-1):120]
predators = xr.concat([predators, xr.zeros_like(predators.isel(time = 0))], 
                      dim = 'time')
predators['time'] = time

#set initial consumer size spectrum 
detritivores = uf.init_da(log10_size_bins, time)
detritivores.loc[{'size_class': log10_size_bins[(ind_min_detritivore_size-1):120],
                 'time': detritivores.time.min()}] = params['detritivore_sizes'][(ind_min_detritivore_size-1):120]

#set initial detritivore spectrum
detritus = xr.DataArray(data = np.append(np.array(params['init_detritus']), 
                                         np.repeat(np.array(0),numb_time_steps)),
                        dims = 'time', coords = {'time': time})

if use_init:
    predators.loc[{'size_class': log10_size_bins[(ind_min_pred_size-1):],
                   'time': predators.time.min()}] = params['plank_pred_sizes'][(ind_min_pred_size-1):]
    detritivores.loc[{'size_class': log10_size_bins[(ind_min_detritivore_size-1):],
                 'time': detritivores.time.min()}] = params['detritivore_sizes'][(ind_min_detritivore_size-1):]

In [117]:
other_mort_pred = xr.DataArray(data = params['natural_mort']*10**(-0.25*log10_size_bins),
                               dims = ['size_class'], 
                               coords = {'size_class': log10_size_bins})
other_mort_det = params['natural_mort']*10**(-0.25*log10_size_bins_mat)
senes_mort_det = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins_mat-params['size_senescence'])))
senes_mort_pred = (params['const_senescence_mort']*
                   10**(params['exp_senescence_mort']*
                        (log10_size_bins_mat-params['size_senescence'])))

In [19]:
fishing_mort_pred = xr.DataArray(data = np.repeat(
    np.array(params['effort'])*params['fish_mort_pred'], numb_size_bins).\
                                 reshape(numb_size_bins, numb_time_steps),
                                 dims = ['size_class', 'time'], 
                                 coords = {'size_class': log10_size_bins,
                                           'time': dbpm_input_time})

fishing_mort_pred = xr.where(
    (fishing_mort_pred.size_class >= log10_size_bins[int(ind_min_fish_pred-1)]) &
    (fishing_mort_pred.size_class < fishing_mort_pred.size_class.max()),
    fishing_mort_pred, 0)

In [20]:
fishing_mort_det = xr.DataArray(data = np.repeat(
    np.array(params['effort'])*params['fish_mort_detritivore'], numb_size_bins).\
                                 reshape(numb_size_bins, numb_time_steps),
                                 dims = ['size_class', 'time'], 
                                 coords = {'size_class': log10_size_bins,
                                           'time': dbpm_input_time})

fishing_mort_det = xr.where(
    (fishing_mort_det.size_class >= log10_size_bins[int(ind_min_fish_det-1)]) &
    (fishing_mort_det.size_class < fishing_mort_det.size_class.max()),
    fishing_mort_det, 0)

In [21]:
catch_pred = fishing_mort_pred*size_bin_vals
catch_pred = xr.concat([catch_pred.isel(time = 0), catch_pred], dim = 'time')
catch_pred['time'] = time
catch_pred = xr.where((catch_pred.size_class >= log10_size_bins[int(ind_min_fish_pred-1)]) &
                (catch_pred.size_class < max(log10_size_bins))&
         (catch_pred.time == min(time)),
         catch_pred*predators, 0)

In [22]:
catch_det = fishing_mort_det*size_bin_vals
catch_det = xr.concat([catch_det.isel(time = 0), catch_det], dim = 'time')
catch_det['time'] = time
catch_det = xr.where((catch_det.size_class >= log10_size_bins[int(ind_min_fish_det-1)]) &
                     (catch_det.size_class < max(log10_size_bins)) &
                     (catch_det.time == min(time)), catch_det*detritivores, 0)

In [23]:
if temp_effect:
    #Adding time dimension to temperature effect for pelagic group
    pel_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
                                                (params['boltzmann']*
                                                 (np.array(params['sea_surf_temp'])+273))),
                                  dims = ['time'],
                                  coords = {'time': dbpm_input_time})
    
    #Adding time dimension to temperature effect for benthic group
    ben_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
                                                (params['boltzmann']*
                                                 (np.array(params['sea_floor_temp'])+273))),
                                  dims = ['time'],
                                  coords = {'time': dbpm_input_time})

In [24]:
feed_mult_pel = (params['hr_volume_search']*
                 10**(log10_size_bins_mat*metabolic_req_pred)*
                 params['pref_pelagic'])

feed_mult_ben = (params['hr_volume_search']*
                 10**(log10_size_bins_mat*metabolic_req_pred)*
                 params['pref_benthos'])

growth_prop = 1-np.array(params['defecate_prop'])
high_prop = 1-np.array(params['def_low'])

In [25]:
i = 0
t = dbpm_input.time[i]
ts = time[i]

In [179]:
pred_growth = (feed_mult_pel*
               np.dot((predators.sel(time = ts)*log_size_increase),
                      constant_growth))
feed_rate_pel = (pel_tempeffect.sel(time = t)*
                 (pred_growth/(1+params['handling']*pred_growth)))

detrit_growth = (feed_mult_ben*
                 np.dot((detritivores.sel(time = ts)*log_size_increase),
                        constant_growth))

feed_rate_bent = (pel_tempeffect.sel(time = t)*
                 (detrit_growth/(1+params['handling']*detrit_growth)))

detritus_multiplier = ((1/size_bin_vals)*params['hr_vol_filter_benthos']*
                       10**(log10_size_bins_mat*metabolic_req_detritivore)*
                       detritus.sel(time = ts))
feed_rate_det = (ben_tempeffect.sel(time = t)*detritus_multiplier/
                 (1+params['handling']*detritus_multiplier))

In [27]:
growth_int_pred.loc[{'time': ts}] = (growth_prop*params['growth_pred']*
                                     feed_rate_pel+high_prop*
                                     params['growth_detritivore']*feed_rate_bent)

In [28]:
reprod_pred.loc[{'time': ts}] = (growth_prop*
                                 (1-(np.array(params['growth_pred'])+
                                     params['energy_pred']))*
                                 feed_rate_pel+growth_prop*
                                 (1-(np.array(params['growth_detritivore'])+
                                     params['energy_detritivore']))*
                                 feed_rate_bent)

In [29]:
sat_pel = xr.where(feed_rate_pel > 0, feed_rate_pel/pred_growth, 0)

In [184]:
pred_mort_pred.loc[{'time': ts}] = ((params['pref_pelagic']*met_req_log10_size_bins*
                                    params['hr_volume_search'])*
                                    np.dot((predators.sel(time = ts)*sat_pel*
                                            log_size_increase), 
                                           constant_mortality))

tot_mort_pred.loc[{'time': ts}] = (pred_mort_pred.sel(time = ts)+
                                   pel_tempeffect.sel(time = t)*
                                   other_mort_pred+senes_mort_pred+
                                   fishing_mort_pred.sel(time = t))

In [31]:
growth_det.loc[{'time': ts}] = high_prop*params['growth_detritus']*feed_rate_det
reprod_det.loc[{'time': ts}] = (high_prop*(1-(np.array(params['growth_detritus'])+
                                              params['energy_detritivore']))*
                                feed_rate_det)

In [186]:
divisor = ((params['hr_volume_search']*
            10**(log10_size_bins_mat*metabolic_req_detritivore)*
            params['pref_benthos'])*
           np.dot((detritivores.sel(time = ts)*log_size_increase), 
                                     constant_growth))
sat_ben = xr.where(feed_rate_bent > 0, feed_rate_bent/divisor, 0)

pred_mort_det.loc[{'time': ts}] = xr.where(sat_ben > 0, 
                                           ((params['pref_benthos']*
                                             met_req_log10_size_bins*
                                             params['hr_volume_search'])*
                                            np.dot((predators.sel(time = ts)*
                                                    sat_ben*log_size_increase),
                                                   constant_mortality)), 0)

tot_mort_det.loc[{'time': ts}] = (pred_mort_det.sel(time = ts)+
                                  ben_tempeffect.sel(time = t)*other_mort_det+
                                  senes_mort_det+fishing_mort_det.sel(time = t))

In [187]:
output_w = (size_bin_vals*feed_rate_det*detritivores.sel(time = ts)*
            log_size_increase).sum()

defbypred = (params['defecate_prop']*feed_rate_pel*size_bin_vals*
             predators.sel(time = ts)+params['def_low']*feed_rate_bent*
             size_bin_vals*predators.sel(time = ts))

In [188]:
input_w = (sinking_rate.sel(time = t)* 
                   ((defbypred.isel(size_class = slice(ind_min_pred_size-1, 
                                                       numb_size_bins))*
                     log_size_increase).sum()+
                    (pel_tempeffect.sel(time = t)*other_mort_pred*
                     predators.sel(time = ts)*size_bin_vals*
                     log_size_increase).sum()+ 
                    (pel_tempeffect.sel(time = t)*senes_mort_pred*
                     predators.sel(time = ts)*size_bin_vals*
                     log_size_increase).sum())+
                   ((ben_tempeffect.sel(time = t)*other_mort_det*
                     detritivores.sel(time = ts)*size_bin_vals*
                     log_size_increase).sum()+ 
                    (ben_tempeffect.sel(time = t)*senes_mort_det*
                     detritivores.sel(time = ts)*size_bin_vals*
                     log_size_increase).sum()))

burial = input_w*(0.013+0.53*input_w**2/(7+input_w)**2)

dW = input_w-(output_w+burial)

detritus.loc[{'time': t}] = detritus.sel(time = ts)+dW*timesteps_years

In [189]:
Ai_u = np.zeros(numb_size_bins)
Bi_u = np.zeros(numb_size_bins)
Si_u = np.zeros(numb_size_bins)

Ai_u[np.array(params['idx'])-1] = ((1/np.log(10))*
                                   -growth_int_pred.isel(size_class = slice(None, -1)).
                                   sel(time = ts)*timesteps_years/log_size_increase)
Bi_u[np.array(params['idx'])-1] = (1+(1/np.log(10))*
                                   growth_int_pred.isel(size_class = slice(1, None)).
                                   sel(time = ts)*timesteps_years/log_size_increase+
                                   tot_mort_pred.isel(size_class = slice(1, None)).
                                   sel(time = ts)*timesteps_years)
Si_u[np.array(params['idx'])-1] = predators.isel(size_class = slice(1, None)).sel(time = ts)

Ai_u[(ind_min_pred_size-1)] = 0
Bi_u[(ind_min_pred_size-1)] = 1
Si_u[(ind_min_pred_size-1)] = (predators.isel(size_class = (ind_min_pred_size-1)).
                               sel(time = ts))

In [199]:
predators.isel(size_class = ind_min_pred_size-1).\
            loc[{'time': t}] = (predators.isel(size_class = ind_min_pred_size-1).\
                                sel(time = ts)+
                                ((reprod_pred*size_bin_vals*predators*
                                 log_size_increase).sel(time = ts).
                                 isel(size_class = slice(ind_min_pred_size, None)).
                                 sum()*timesteps_years)/
                                (log_size_increase*
                                 size_bin_vals.isel(size_class = ind_min_pred_size-1))-
                                (timesteps_years/log_size_increase)*(1/np.log(10))*
                                ((growth_int_pred*predators).
                                 isel(size_class = ind_min_pred_size-1).sel(time = ts))-
                                timesteps_years*
                                ((tot_mort_pred*predators).
                                 isel(size_class = ind_min_pred_size-1).sel(time = ts)))

for j in range(ind_min_pred_size, numb_size_bins):
        predators.isel(size_class = j).\
        loc[{'time': t}] = ((Si_u[j]-Ai_u[j]*predators.isel(size_class = j-1).\
                             sel(time = t))/Bi_u[j])

In [200]:
Ai_v = np.zeros(numb_size_bins)
Bi_v = np.zeros(numb_size_bins)
Si_v = np.zeros(numb_size_bins)

#shorthand for matrix referencing
Ai_v[idx_new] = ((1/np.log(10))*
                  -growth_det.isel(size_class = slice(ind_min_detritivore_size-1, -1)).
                  sel(time = ts)*timesteps_years/log_size_increase)
Bi_v[idx_new] = (1+(1/np.log(10))*
                 growth_det.isel(size_class = slice(ind_min_detritivore_size, None)).
                 sel(time = ts)*timesteps_years/log_size_increase+
                 tot_mort_det.isel(size_class = slice(ind_min_detritivore_size, None)).
                 sel(time = ts)*timesteps_years)
Si_v[idx_new] = (detritivores.isel(size_class = slice(ind_min_detritivore_size, None)).
                 sel(time = ts))

#boundary condition at upstream end
Ai_v[ind_min_detritivore_size-1] = 0
Bi_v[ind_min_detritivore_size-1] = 1
Si_v[ind_min_detritivore_size-1] = (detritivores.
                                    isel(size_class = ind_min_detritivore_size-1).
                                    sel(time = ts))

In [33]:
(detritivores.isel(size_class = slice(None, ind_min_detritivore_size)).
 loc[{'time': t}]) = (detritivores.isel(size_class = slice(None, ind_min_detritivore_size)).
                      sel(time = ts))

In [212]:
(detritivores.isel(size_class = ind_min_detritivore_size-1).
 loc[{'time': t}]) = (detritivores.isel(size_class = ind_min_detritivore_size-1).
                      sel(time = ts)+
                      ((reprod_det*size_bin_vals*detritivores*log_size_increase).
                       isel(size_class = slice(ind_min_detritivore_size, None)).
                       sel(time = ts).sum()*timesteps_years)/
                      (log_size_increase*
                       size_bin_vals.isel(size_class = ind_min_detritivore_size-1))-
                      (timesteps_years/log_size_increase)*(1/np.log(10))*
                      ((growth_det*detritivores).
                       isel(size_class = ind_min_detritivore_size-1).sel(time = ts))-
                      timesteps_years*
                      ((tot_mort_det*detritivores).
                       isel(size_class = ind_min_detritivore_size-1).sel(time = ts)))

In [213]:
for j in idx_new:
            detritivores.isel(size_class = j).\
            loc[{'time': t}] = ((Si_v[j]-Ai_v[j]*detritivores.isel(size_class = j-1).\
                                 sel(time = t))/Bi_v[j])

In [216]:
catch_all = xr.concat([xr.zeros_like(catch_pred.isel(time = 0)),
              (fishing_mort_pred*predators*(10**log10_size_bins_mat))], dim = 'time')
catch_pred = xr.where((catch_pred.size_class >= log10_size_bins[int(ind_min_fish_pred-1)]) &
         (catch_pred.size_class < max(log10_size_bins)) &
         (catch_pred.time > catch_pred.time.min()), catch_all, catch_pred)

In [214]:
catch_all = xr.concat([xr.zeros_like(catch_det.isel(time = 0)),
                       (fishing_mort_det*detritivores*size_bin_vals)], dim = 'time')
catch_det = xr.where((catch_det.size_class >= log10_size_bins[int(ind_min_fish_det-1)]) &
         (catch_det.size_class < max(log10_size_bins)) &
         (catch_det.time > catch_det.time.min()), catch_all, catch_det)
catch_det

<xarray.DataArray (size_class: 181, time: 13)> Size: 19kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * size_class  (size_class) float64 1kB -12.0 -11.9 -11.8 -11.7 ... 5.8 5.9 6.0
  * time        (time) datetime64[ns] 104B 1959-12-01 1960-01-01 ... 1960-12-01

In [215]:
catch_det[int(ind_min_fish_det-1): , :3]

<xarray.DataArray (size_class: 45, time: 3)> Size: 1kB
array([[0.00000000e+000, 4.23698312e-153, 0.00000000e+000],
       [0.00000000e+000, 3.67514364e-162, 0.00000000e+000],
       [0.00000000e+000, 3.00824836e-171, 0.00000000e+000],
       [0.00000000e+000, 2.32358336e-180, 0.00000000e+000],
       [0.00000000e+000, 1.69352061e-189, 0.00000000e+000],
       [0.00000000e+000, 1.16464029e-198, 0.00000000e+000],
       [0.00000000e+000, 7.55688050e-208, 0.00000000e+000],
       [0.00000000e+000, 4.62617276e-217, 0.00000000e+000],
       [0.00000000e+000, 2.67182204e-226, 0.00000000e+000],
       [0.00000000e+000, 1.45571397e-235, 0.00000000e+000],
       [0.00000000e+000, 7.48173594e-245, 0.00000000e+000],
       [0.00000000e+000, 3.62710283e-254, 0.00000000e+000],
       [0.00000000e+000, 1.65851793e-263, 0.00000000e+000],
       [0.00000000e+000, 7.15242133e-273, 0.00000000e+000],
       [0.00000000e+000, 2.90888437e-282, 0.00000000e+000],
       [0.00000000e+000, 1.11559357e-291, 0.00000000e+000],
       [0.00000000e+000, 4.03417918e-301, 0.00000000e+000],
       [0.00000000e+000, 1.37541914e-310, 0.00000000e+000],
       [0.00000000e+000, 4.96437161e-320, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
...
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])
Coordinates:
  * size_class  (size_class) float64 360B 1.6 1.7 1.8 1.9 ... 5.7 5.8 5.9 6.0
  * time        (time) datetime64[ns] 24B 1959-12-01 1960-01-01 1960-02-01

In [312]:
t

180